In [164]:
import pandas as pd
from utils import *


In [174]:
ingredients_df = pd.read_csv("/Users/Ben/PycharmProjects/froova/ingredients_list.csv")

In [175]:
import openai
import pandas as pd

def prompt_for_replacements(ingredients_df):
    if "Can_not_eat" in ingredients_df.columns and ingredients_df['Can_not_eat'].any():
        # Filter to only those ingredients that cannot be eaten
        restricted_ingredients = ingredients_df[ingredients_df['Can_not_eat'] == True]

        for index, row in restricted_ingredients.iterrows():
            original_ingredient = row['Name']
            # Prompt GPT for a single replacement suggestion
            prompt_text = f"Provide a single ingredient replacement for {original_ingredient}. Make sure you only respond with the name of the substitution only, do not say anything else or else. Also make sure the state of matter of the ingredient is not changed. For example dont, replace eggs with a solid, replace it with something else that is liquid"
            response = openai.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "system", "content": "You are a knowledgeable chef tasked with suggesting a single ingredient replacement."},
                          {"role": "user", "content": prompt_text}]
            )
            suggested_replacement = response.choices[0].message.content.strip()

            # Ask user for approval to replace
            user_approval = input(f"Do you want to replace {original_ingredient} with {suggested_replacement}? (Yes/No): ")
            if user_approval.lower() == 'yes':
                # Prompt GPT for volume and weight if the replacement is approved
                volume_weight_prompt = f"Provide the volume and weight for {suggested_replacement}, formatted as 'volume, weight'."
                volume_weight_response = openai.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=[{"role": "system", "content": "You are a knowledgeable chef tasked with providing the volume and weight of an ingredient formatted as 'volume, weight'. Do not include any extra information."},
                              {"role": "user", "content": volume_weight_prompt}]
                )
                volume_and_weight = volume_weight_response.choices[0].message.content.strip()

                # Split the response into volume and weight
                if ", " in volume_and_weight:
                    volume, weight = volume_and_weight.split(", ")
                    ingredients_df.at[index, 'Name'] = suggested_replacement
                    ingredients_df.at[index, 'Volume'] = volume
                    ingredients_df.at[index, 'Weight'] = weight
                    ingredients_df.at[index, 'Can_not_eat'] = False
                else:
                    # If the format is not as expected, retain the existing volume and weight
                    ingredients_df.at[index, 'Name'] = suggested_replacement
                    ingredients_df.at[index, 'Can_not_eat'] = False

        # Save updated DataFrame to CSV file
        ingredients_df.to_csv('updated_ingredients_list.csv', index=False)
        print("Updated ingredients list saved to 'updated_ingredients_list.csv'.")
    else:
        print("No ingredients need to be replaced due to dietary restrictions.")

    return ingredients_df


In [176]:
x = prompt_for_replacements(ingredients_df)
x.to_csv('updated_ingredients_list.csv')

Updated ingredients list saved to 'updated_ingredients_list.csv'.


In [168]:
pd.read_csv("updated_ingredients_list.csv")

,Unnamed: 0,Name,Volume,Weight,Can_not_eat
0,0,all-purpose flour,2 and 1/4 cups,280g,False
1,1,baking soda,1 teaspoon,NaN,False
2,2,cornstarch,1 and 1/2 teaspoons,NaN,False
3,3,salt,1/2 teaspoon,NaN,False
4,4,Coconut oil,1 cup,218 g,False
5,5,light or dark brown sugar,3/4 cup,150g,False
6,6,granulated sugar,1/2 cup,100g,False
7,7,Flaxseed meal mixed with water,1 cup,120 grams,False
8,8,Silken tofu,1 cup,240 grams,False
9,9,pure vanilla extract,2 teaspoons,NaN,False


In [211]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import csv
import time

# Initialize Chrome WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# URL for Sprouts search page for "flour"
url = "https://shop.sprouts.com/search?search_term=flour&search_is_autocomplete=false"

# Open the URL in the browser
driver.get(url)

# Wait for the page to fully load
time.sleep(5)  # You can adjust this if needed

# Get the page source and parse it with BeautifulSoup
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Close the browser after fetching the content
driver.quit()

# List to store product data
products = []

# Find all product cards
product_cards = soup.find_all('li', class_='product-wrapper')

# Loop through each product card and extract the required information
for card in product_cards:
    product = {}

    # Extract the product name
    product_name_tag = card.find('div', class_='css-15uwigl')
    if product_name_tag:
        product['name'] = product_name_tag.text.strip()
    else:
        product['name'] = 'N/A'  # If the name is not found, mark it as 'N/A'

    # Extract the product price
    product_price_tag = card.find('span', class_='css-coqxwd')
    if product_price_tag:
        product['price'] = product_price_tag.text.strip()
    else:
        product['price'] = 'N/A'  # If the price is not found, mark it as 'N/A'

    # Extract the product image URL
    product_image_tag = card.find('img', class_='css-1eer7o2')
    if product_image_tag and 'src' in product_image_tag.attrs:
        product['image_url'] = product_image_tag['src']
    else:
        product['image_url'] = 'N/A'  # If the image is not found, mark it as 'N/A'

    # Add the product to the list if any fields were found
    products.append(product)

# Save the extracted data to a CSV file
filename = 'sprouts_flour_products.csv'
with open(filename, 'w', newline='', encoding='utf-8') as f:
    w = csv.DictWriter(f, fieldnames=['name', 'price', 'image_url'])
    w.writeheader()
    for product in products:
        w.writerow(product)

print(f"Data has been saved to {filename}")


Data has been saved to sprouts_flour_products.csv
